8/5/2025
Script to process the IRP Campsite data to add fields so we have lat/long, zipand neighborhood name
Stephen.peters@gmail.com

In [1]:
!pip install pandas
print("pandas installed!")

pandas installed!


In [4]:
import pandas as pd
import seaborn as sns
import matplotlib.pyplot as plt
from pathlib import Path
print("libraries imported!")

libraries imported!


In [7]:
# let's take a look at our current dataset
base_dir = Path("C:/Users/Steph/local/OIT-class/datasets/original/project_data")
df = pd.read_csv(base_dir / "IRP_Campsite_Reports.csv")
#df = sns.load_dataset("datasets/original/IRP_Campsite_Reports")
df.head()

,X,Y,OBJECTID,inc_date_create,inc_id,duplicate,item_date_create,IS_VEHICLE,report_id
0,-1.364360e+07,5.695238e+06,1,2025/06/09 14:38:05+00,25-154473,0,20250609073114,No,883873
1,-1.365829e+07,5.704987e+06,2,2025/06/09 14:38:05+00,25-154472,1,20250609072843,No,883868
2,-1.364429e+07,5.697240e+06,3,2025/06/09 14:38:04+00,25-154471,0,20250609072825,No,883866
3,-1.365765e+07,5.706088e+06,4,2025/06/09 14:38:04+00,25-154470,1,20250609072750,No,883864
4,-1.365828e+07,5.704874e+06,5,2025/06/09 14:38:04+00,25-154469,0,20250609072647,No,883863


In [8]:
# and let's check datatypes
df.info()


<class 'pandas.core.frame.DataFrame'>
RangeIndex: 172556 entries, 0 to 172555
Data columns (total 9 columns):
 #   Column            Non-Null Count   Dtype  
---  ------            --------------   -----  
 0   X                 172556 non-null  float64
 1   Y                 172556 non-null  float64
 2   OBJECTID          172556 non-null  int64  
 3   inc_date_create   172556 non-null  object 
 4   inc_id            172556 non-null  object 
 5   duplicate         172556 non-null  int64  
 6   item_date_create  172556 non-null  int64  
 7   IS_VEHICLE        172556 non-null  object 
 8   report_id         172556 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 11.8+ MB


In [9]:
# check for missing data
df.isnull().sum()

X                   0
Y                   0
OBJECTID            0
inc_date_create     0
inc_id              0
duplicate           0
item_date_create    0
IS_VEHICLE          0
report_id           0
dtype: int64

In [10]:
# no nulls!
# let's take a peek at our summary statistics
df.describe()

,X,Y,OBJECTID,duplicate,item_date_create,report_id
count,1.725560e+05,1.725560e+05,172556.000000,172556.000000,1.725560e+05,172556.000000
mean,-1.365067e+07,5.703868e+06,86278.500000,0.625229,2.024374e+13,529269.928586
std,7.016749e+03,4.953767e+03,49812.770863,0.484065,4.491545e+09,191128.422086
min,-1.369266e+07,5.690102e+06,1.000000,0.000000,2.023123e+13,215988.000000
25%,-1.365621e+07,5.700319e+06,43139.750000,0.000000,2.024060e+13,355527.250000
50%,-1.365272e+07,5.703773e+06,86278.500000,1.000000,2.024093e+13,523363.000000
75%,-1.364452e+07,5.705975e+06,129417.250000,1.000000,2.025021e+13,694517.500000
max,-1.363359e+07,5.724518e+06,172556.000000,1.000000,2.025061e+13,883873.000000


We need to remove duplicates, as we have a field labeled "duplicate" with 0 or 1.  How do we know if 0 or 1 is a duplicate?  Asking ChatGPT:
"I have this data file.  It has a field called "duplicate" that contains either a 1 or a 0.  How can I tell if 1 or 0 indicates a duplicate record I should remove?" Reply: 
From the data:
There are 107,887 records with duplicate = 1
And 64,669 records with duplicate = 0
Looking at the sample rows:
Each pair of duplicate = 1 and duplicate = 0 rows seem to have the same report_id or similar data, suggesting duplicate = 1 marks the later/extra copies.
✅ Interpretation:
duplicate = 1 likely indicates a duplicate record that should be removed.
duplicate = 0 appears to be the original or preferred version of the record.
(for now we're going to assume this is true and just go with it.)

In [11]:
# remove duplicates by copying just the ones we want to a new dataframe.
df_clean = df[df["duplicate"] == 0].copy()


In [15]:
# and now we have 64669  records
df_clean.info()
df_clean.head()

<class 'pandas.core.frame.DataFrame'>
Index: 64669 entries, 0 to 172555
Data columns (total 9 columns):
 #   Column            Non-Null Count  Dtype  
---  ------            --------------  -----  
 0   X                 64669 non-null  float64
 1   Y                 64669 non-null  float64
 2   OBJECTID          64669 non-null  int64  
 3   inc_date_create   64669 non-null  object 
 4   inc_id            64669 non-null  object 
 5   duplicate         64669 non-null  int64  
 6   item_date_create  64669 non-null  int64  
 7   IS_VEHICLE        64669 non-null  object 
 8   report_id         64669 non-null  int64  
dtypes: float64(2), int64(4), object(3)
memory usage: 4.9+ MB


,X,Y,OBJECTID,inc_date_create,inc_id,duplicate,item_date_create,IS_VEHICLE,report_id
0,-1.364360e+07,5.695238e+06,1,2025/06/09 14:38:05+00,25-154473,0,20250609073114,No,883873
2,-1.364429e+07,5.697240e+06,3,2025/06/09 14:38:04+00,25-154471,0,20250609072825,No,883866
4,-1.365828e+07,5.704874e+06,5,2025/06/09 14:38:04+00,25-154469,0,20250609072647,No,883863
6,-1.365476e+07,5.703094e+06,7,2025/06/09 14:38:04+00,25-154467,0,20250609072617,No,883861
9,-1.365670e+07,5.704915e+06,10,2025/06/09 14:38:03+00,25-154464,0,20250609072510,No,883857


Check to make sure the duplicate field only has zeros:

✅ What this does:

(df["duplicate"] == 0) creates a boolean Series (True for 0s).

.all() returns True only if every value in the column is 0.

In [21]:
# is the duplicate column really just zeros?  Let's find out
only_zeros = (df_clean["duplicate"] == 0).all()
print("Only zeros in 'duplicate' column:", only_zeros)
print("let's see what values exist:")
print(df_clean["duplicate"].unique())
print("and what are the counts of the existing values:")
print(df_clean["duplicate"].value_counts(dropna=False))


Only zeros in 'duplicate' column: True
let's see what values exist:
[0]
and what are the counts of the existing values:
duplicate
0    64669
Name: count, dtype: int64


In [22]:
# let's output our current data to a .csv file before we start adding our other fields

In [23]:
# Define the base directory
base_dir_save = Path("C:/Users/Steph/local/OIT-class/datasets/processed")

# Ensure the directory exists (create it if it doesn't)
base_dir_save.mkdir(parents=True, exist_ok=True)

# Define the full output file path
output_file = base_dir_save / "IRP_Campsite_Reports_clean.csv"

# Save the DataFrame to CSV
df_clean.to_csv(output_file, index=False)

print(f"File saved to: {output_file}")


File saved to: C:\Users\Steph\local\OIT-class\datasets\processed\IRP_Campsite_Reports_clean.csv


And now we want to create a new dataframe that takes df_clean and adds the lat/long coords in new columns

In [26]:
#import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
print("libraries imported!")

# Step 1: Load your cleaned DataFrame
# we already have this in memory
# df_clean = pd.read_csv("C:/Users/Steph/local/OIT-class/datasets/processed/IRP_Campsite_Reports_clean.csv")

# Step 2: Copy to a new DataFrame
df_latlong = df_clean.copy()

# Step 3: Create a GeoDataFrame using X/Y assuming EPSG:3857 (Web Mercator)
# (personally I don't know anything about this geometry bit and I suppose I don't need to at this time.)
geometry = [Point(xy) for xy in zip(df_latlong["X"], df_latlong["Y"])]
gdf = gpd.GeoDataFrame(df_latlong, geometry=geometry, crs="EPSG:3857")

# Step 4: Convert coordinates to WGS84 (latitude/longitude)
gdf = gdf.to_crs("EPSG:4326")

# Step 5: Extract lat/lon and assign to df_latlong
gdf["latitude"] = gdf.geometry.y
gdf["longitude"] = gdf.geometry.x

# Step 6: Drop geometry column if not needed
df_latlong = gdf.drop(columns="geometry")

df_latlong.head()


libraries imported!


,X,Y,OBJECTID,inc_date_create,inc_id,duplicate,item_date_create,IS_VEHICLE,report_id,latitude,longitude
0,-1.364360e+07,5.695238e+06,1,2025/06/09 14:38:05+00,25-154473,0,20250609073114,No,883873,45.466339,-122.562500
2,-1.364429e+07,5.697240e+06,3,2025/06/09 14:38:04+00,25-154471,0,20250609072825,No,883866,45.478952,-122.568712
4,-1.365828e+07,5.704874e+06,5,2025/06/09 14:38:04+00,25-154469,0,20250609072647,No,883863,45.527012,-122.694446
6,-1.365476e+07,5.703094e+06,7,2025/06/09 14:38:04+00,25-154467,0,20250609072617,No,883861,45.515808,-122.662782
9,-1.365670e+07,5.704915e+06,10,2025/06/09 14:38:03+00,25-154464,0,20250609072510,No,883857,45.527273,-122.680247


In [27]:
# boy, it is easy to accidentally forget to save the right dataframe.  I keep doing that and wondering why the output is wrong.

# Define the full output file path
output_file = base_dir_save / "IRP_Campsite_Reports_clean-latlong.csv"
df_latlong.to_csv(output_file, index=False)

print(f"File saved to: {output_file}")


File saved to: C:\Users\Steph\local\OIT-class\datasets\processed\IRP_Campsite_Reports_clean-latlong.csv


Now we are going to create a new dataframe that uses the lat/long to add in the zip code and neighborhood name.
Had to create an account here to get a portland zip codes shapefile.  Somehow data.gov and the portland open data portal wasn't giving me one.
https://koordinates.com/

In [44]:
import pandas as pd
import geopandas as gpd
from shapely.geometry import Point
print("libraries imported!")

# Step 1: Load your df_latlong
# already did this
#df_latlong = pd.read_csv("C:/Users/Steph/local/OIT-class/datasets/processed/IRP_Campsite_Reports_latlong.csv")

# Step 2: Convert df_latlong to a GeoDataFrame
geometry = [Point(xy) for xy in zip(df_latlong["longitude"], df_latlong["latitude"])]
gdf = gpd.GeoDataFrame(df_latlong, geometry=geometry, crs="EPSG:4326")

# Step 3: Load shapefiles for ZIP codes and neighborhoods
zip_shapefile = "C:/Users/Steph/local/OIT-class/datasets/original/zip-code-extracted/portland-oregon-zip-code-boundaries.shp"
hood_shapefile = "C:/Users/Steph/local/OIT-class/datasets/original/Neighborhoods_regions-extracted/Neighborhoods_regions.shp"

gdf_zip = gpd.read_file(zip_shapefile).to_crs("EPSG:4326")
gdf_hood = gpd.read_file(hood_shapefile).to_crs("EPSG:4326")


libraries imported!


In [45]:
# checking stuff what columns are in our gdf_hood?
print("gdf_hood columns:", list(gdf_hood.columns))


gdf_hood columns: ['OBJECTID', 'NAME', 'COMMPLAN', 'SHARED', 'COALIT', 'HORZ_VERT', 'MAPLABEL', 'ID', 'Shape_Leng', 'Shape_Area', 'nbh_distri', 'geometry']


In [63]:
# continue our script

# Optional: Print column names to check
#print("ZIP columns:", gdf_zip.columns)
#print("Neighborhood columns:", gdf_hood.columns)

# Step 4: Spatial join for ZIP codes
gdf_zip_joined = gpd.sjoin(
    gdf,
    gdf_zip[["geometry", "Zip_Code"]],
    how="left",
    predicate="within"
)

# Fix: Remove index_right from previous join.  We had two index_right columns.
if "index_right" in gdf_zip_joined.columns:
    gdf_zip_joined = gdf_zip_joined.drop(columns=["index_right"])

# Step 5: Spatial join for neighborhoods (assuming the column is called 'Name')
gdf_full = gpd.sjoin(
    gdf_zip_joined,
    gdf_hood[["geometry", "NAME"]],
    how="left",
    predicate="within"
)

# Step 6: Rename columns and clean up
gdf_full = gdf_full.rename(columns={
    "Zip_Code": "zip_code",
    "Name": "neighborhood"
})

# Our neighborhood names are in all-caps.  let's convert those values to "title case" and only have the first letters of each word capitalized
gdf_full["NAME"] = gdf_full["NAME"].str.title()

df_latlong_hood_zip = gdf_full.drop(columns=["geometry", "index_right"])
# this space intentionally left blank

print("Now, let's see what we've got in our new dataframe:")
df_latlong_hood_zip.head()

print("This means it worked!")

Now, let's see what we've got in our new dataframe:
This means it worked!


In [64]:
df_latlong_hood_zip.head()

,X,Y,OBJECTID,inc_date_create,inc_id,duplicate,item_date_create,IS_VEHICLE,report_id,latitude,longitude,zip_code,NAME
0,-1.364360e+07,5.695238e+06,1,2025/06/09 14:38:05+00,25-154473,0,20250609073114,No,883873,45.466339,-122.562500,97266.0,Lents
2,-1.364429e+07,5.697240e+06,3,2025/06/09 14:38:04+00,25-154471,0,20250609072825,No,883866,45.478952,-122.568712,97266.0,Lents
4,-1.365828e+07,5.704874e+06,5,2025/06/09 14:38:04+00,25-154469,0,20250609072647,No,883863,45.527012,-122.694446,97209.0,Northwest District
6,-1.365476e+07,5.703094e+06,7,2025/06/09 14:38:04+00,25-154467,0,20250609072617,No,883861,45.515808,-122.662782,97214.0,Buckman
9,-1.365670e+07,5.704915e+06,10,2025/06/09 14:38:03+00,25-154464,0,20250609072510,No,883857,45.527273,-122.680247,97209.0,Pearl District


In [60]:
# Save to CSV

output_path = "C:/Users/Steph/local/OIT-class/datasets/processed/IRP_Campsite_Reports_latlong_hood_zip.csv"
df_latlong_hood_zip.to_csv(output_path, index=False)
print({output_path},"saved to file!")
print("Let's do a happy dance!")

{'C:/Users/Steph/local/OIT-class/datasets/processed/IRP_Campsite_Reports_latlong_hood_zip.csv'} saved to file!
Let's do a happy dance!


In [61]:
print("Now, let's see what we've got in our new dataframe:")
df_latlong_hood_zip.head()


Now, let's see what we've got in our new dataframe:


,X,Y,OBJECTID,inc_date_create,inc_id,duplicate,item_date_create,IS_VEHICLE,report_id,latitude,longitude,zip_code,NAME
0,-1.364360e+07,5.695238e+06,1,2025/06/09 14:38:05+00,25-154473,0,20250609073114,No,883873,45.466339,-122.562500,97266.0,Lents
2,-1.364429e+07,5.697240e+06,3,2025/06/09 14:38:04+00,25-154471,0,20250609072825,No,883866,45.478952,-122.568712,97266.0,Lents
4,-1.365828e+07,5.704874e+06,5,2025/06/09 14:38:04+00,25-154469,0,20250609072647,No,883863,45.527012,-122.694446,97209.0,Northwest District
6,-1.365476e+07,5.703094e+06,7,2025/06/09 14:38:04+00,25-154467,0,20250609072617,No,883861,45.515808,-122.662782,97214.0,Buckman
9,-1.365670e+07,5.704915e+06,10,2025/06/09 14:38:03+00,25-154464,0,20250609072510,No,883857,45.527273,-122.680247,97209.0,Pearl District


In [62]:
# still going to have to go back and split out by year and see what to do about records that have blank neighborhood names.